# Lotus SDK

O Lotus SDK é uma solução abrangente e unificada que simplifica o desenvolvimento, o treinamento e a governança de modelos de machine learning. Ele faz parte da plataforma de MLOPs Lotus que entrega soluções para todas as etapas do ciclo de vida de modelos de machine learning. <br> <br>
Ele integra e abstrai diversas funcionalidades de SDKs especializados, como o Cidarta para otimização e treinamento de modelos, o Andromeda para gerenciamento de projetos e experimentos, e o Smith para consumo de features e execução de queries em bases de dados. O SDK Lotus oferece uma interface centralizada e intuitiva, permitindo que os usuários foquem em suas tarefas principais sem se preocupar com a complexidade dos processos subjacentes.

Principais Funcionalidades:

Gerenciamento de Projetos e Experimentos

O SDK Lotus integra-se ao Andromeda, um model registry que gerencia os projetos e experimentos de machine learning.
Os usuários podem criar projetos diretamente no SDK Lotus, que gerencia automaticamente a criação de experimentos antes de cada treinamento, garantindo rastreabilidade e governança completas.

Treinamento de Modelos

O SDK oferece uma interface simples para treinar modelos de machine learning usando o Cidarta.
Suporta treinamento local e na AWS, utilizando o Amazon SageMaker. A escolha entre treinamento local ou na nuvem é totalmente transparente para o usuário.
Antes de cada treinamento, um experimento é automaticamente criado no Andromeda, associando os resultados ao projeto correspondente.

Consumo de Features e Execução de Queries

O SDK Lotus inclui uma poderosa integração com o Smith SDK, permitindo que os usuários consumam features de um catálogo e executem queries diretamente no AWS Athena.
Basta indicar as features desejadas, e o SDK coleta os metadados, monta a query e a executa, retornando os resultados em um pandas DataFrame pronto para análise.

Integração com AWS

O SDK Lotus oferece métodos para login e gerenciamento de credenciais AWS, simplificando o acesso aos recursos na nuvem.
A execução de queries e o treinamento de modelos na AWS são completamente transparentes, permitindo que os usuários foquem nos resultados, sem se preocupar com a configuração da infraestrutura.


In [1]:
from lotus_sdk.lotus import Lotus

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Inicializar o SDK Lotus
lotus = Lotus()


In [3]:
# Resgata credenciais
aws_credentials = lotus.aws_login()
aws_credentials


{'account_credentials': 'account_based_credentials'}

In [4]:
# Recuperar features e executar uma query transparente no Athena
df = lotus.get_features(["feature1", "feature2"])
print("Resultado da query em DataFrame:")
df


Executando query no Athena: SELECT feature1, feature2 FROM table1
Resultado da query em DataFrame:


,feature1,feature2,target
0,value1a,value2a,targeta
1,value1b,value2b,targetb
2,value1c,value2c,targetc


In [5]:
X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [6]:
# Criar um novo projeto no Andromeda
project_id = lotus.create_project(
    name="Project Alpha",
    description="Projeto para classificação de imagens",
    members=["user1", "user2"]
)
print(f"Projeto criado com ID: {project_id}")


Projeto Criado no Andromeda
Projeto criado com ID: 0123456


In [7]:
blueprints = lotus.list_blueprints_pipelines()
blueprints

{'simple_clf': 'Default Classifier with common preprocessing steps',
 'xgboost_clf': 'XGBoost Classifier with common preprocessing steps',
 'xgboost_reg': 'XGBoost Regressor with common preprocessing steps'}

In [8]:
# Executar o treinamento de um modelo com o Cidarta
training_model = lotus.train_model(
    project_id=project_id,
    data_train=df_train.drop(columns=['target']),
    target=df_train['target'],
    blueprint="xgboost_clf",
    run_on_training_job=False
)
print(f"Resultado do treinamento: {training_model}")


Experimento com id 789456, criado no projeto 0123456
Experimento criado automaticamente com ID: 789456
Resultado do treinamento: {'status': 'success', 'model': "<class 'sklearn.pipeline.Pipeline'>", 'experiment_id': '789456'}


In [9]:
model_artifact = training_model['model']
model_artifact

"<class 'sklearn.pipeline.Pipeline'>"

In [10]:
explainer_model = lotus.explainer_model(model_artifact)
explainer_model

'Lotus Explain Model'